### 3ª. Etapa -  Descarte de Valores que não serão utilizado devido a ausência de dados ou dados conflitantes

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Resumo da Tabela Principal
tabelas = pd.read_csv('tabelas_v2.csv')
tabelas.head(3)

,IDADEMAE,GRAVIDEZ,PARTO,SEXO,PESO,IDANOMAL,ANO,FAIXA_ETARIA,CODANOMAL_CID_UNICO
0,25.0,1.0,Normal,Masculino,3800.0,Não,2001,21-30,NaN
1,15.0,1.0,Normal,Masculino,3100.0,Não,2001,0-20,NaN
2,35.0,1.0,Cesária,Masculino,3300.0,Não,2001,31-40,NaN


In [3]:
#Total de Crianças com anomalias com codigo de Anomalia registrado na Coluna CODANOMAL
c_CID_anomalia = tabelas.CODANOMAL_CID_UNICO.value_counts().sum()
s_CID_anomalia = tabelas.CODANOMAL_CID_UNICO.value_counts(dropna=False)[0]
print(f'Total de Crianças com alguma anomalia: {c_CID_anomalia:,.0f}')
print(f'Total de Crianças sem anomalia       : {s_CID_anomalia:,.0f}')

Total de Crianças com alguma anomalia: 424,564
Total de Crianças sem anomalia       : 58,346,722


In [4]:
# Analise de Anomalias confirmadas com base na coluna IDANOMAL
c_anomalia = tabelas.IDANOMAL[tabelas.IDANOMAL=='Sim'].count()
s_anomalia = tabelas.IDANOMAL[tabelas.IDANOMAL=='Não'].count()
print(f'Total de Crianças com alguma anomalia: {c_anomalia:,.0f}')
print(f'Total de Crianças sem anomalia       : {s_anomalia:,.0f}')

Total de Crianças com alguma anomalia: 426,609
Total de Crianças sem anomalia       : 55,622,819


#### 3.1 Idenficamos inconsistência nos dados nas colunas IDANOMAL e CODANOMAL.

In [5]:
# Temos uma quantidade de crianças com anomalia em IDANOMAL mais sem o registro do CID em CODANOMAL
print(f'Total de Crianças marcadas com anomalia em IDANOMAL, mas sem o CID em CODANOMAL_CID_UNICO: {c_anomalia-c_CID_anomalia:,.0f}')
print(f'Total de Crianças com registros nulos ou invalidos ou não informado, no periodo de 2001 a 2020: {s_CID_anomalia-s_anomalia:,.0f}')

Total de Crianças marcadas com anomalia em IDANOMAL, mas sem o CID em CODANOMAL_CID_UNICO: 2,045
Total de Crianças com registros nulos ou invalidos ou não informado, no periodo de 2001 a 2020: 2,723,903


#### 3.2 Com base no que foi demosntrado acima, iremos descartar esses dados Não Infornado e NAN de IDANOMAL.

In [6]:
# Com base no que foi demosntrado acima, iremos descartar esses dados Não Infornado e NAN de IDANOMAL.
tabelas.IDANOMAL.value_counts(dropna=False)

Não              55622819
NaN               1602140
Não Informado     1119718
Sim                426609
Name: IDANOMAL, dtype: int64

In [7]:
tabelas = tabelas.query("IDANOMAL=='Sim' or IDANOMAL=='Não'")
tabelas.IDANOMAL.value_counts(dropna=False)

Não    55622819
Sim      426609
Name: IDANOMAL, dtype: int64

### 3.3 Revisando os ajustes dos dados Não Infornado e NAN de IDANOMAL.

In [8]:
#Total de Crianças com anomalias com codigo de Anomalia registrado na Coluna CODANOMAL
c_CID_anomalia = tabelas.CODANOMAL_CID_UNICO.value_counts().sum()
s_CID_anomalia = tabelas.CODANOMAL_CID_UNICO.value_counts(dropna=False)[0]
print(f'Total de Crianças com alguma anomalia: {c_CID_anomalia:,.0f}')
print(f'Total de Crianças sem anomalia       : {s_CID_anomalia:,.0f}')

Total de Crianças com alguma anomalia: 424,518
Total de Crianças sem anomalia       : 55,624,910


In [9]:
# Analise de Anomalias confirmadas com base na coluna IDANOMAL
c_anomalia = tabelas.IDANOMAL[tabelas.IDANOMAL=='Sim'].count()
s_anomalia = tabelas.IDANOMAL[tabelas.IDANOMAL=='Não'].count()
print(f'Total de Crianças com alguma anomalia: {c_anomalia:,.0f}')
print(f'Total de Crianças sem anomalia       : {s_anomalia:,.0f}')

Total de Crianças com alguma anomalia: 426,609
Total de Crianças sem anomalia       : 55,622,819


In [10]:
# Temos uma quantidade de crianças com anomalia em IDANOMAL mais sem o registro do CID em CODANOMAL
print(f'Total de Crianças marcadas com anomalia em IDANOMAL, mas sem o CID em CODANOMAL_CID_UNICO: {c_anomalia-c_CID_anomalia:,.0f}')
print(f'Total de Crianças que consta sem anomalia em IDANOMAL mas apresentam CID em CODANOMAL: {s_anomalia -s_CID_anomalia:,.0f}')

Total de Crianças marcadas com anomalia em IDANOMAL, mas sem o CID em CODANOMAL_CID_UNICO: 2,091
Total de Crianças que consta sem anomalia em IDANOMAL mas apresentam CID em CODANOMAL: -2,091


#### 3.4 Conciliando os dados das colunas CODANOMAL_CID_UNICO e IDANOMAL.

In [11]:
tabelas.CODANOMAL_CID_UNICO.value_counts(dropna=False).head(3)

NaN     55624910
Q699       35225
Q668       23511
Name: CODANOMAL_CID_UNICO, dtype: int64

In [12]:
tabelas.CODANOMAL_CID_UNICO.value_counts(dropna=False).sum()

56049428

In [13]:
tabelas.IDANOMAL.value_counts(dropna=False)

Não    55622819
Sim      426609
Name: IDANOMAL, dtype: int64

In [14]:
tabelas.IDANOMAL.value_counts(dropna=False).sum()

56049428

In [15]:
# Construindo uma coluna Boleana para fazer o ajuste dos dados, vamos considerar prioridade para ajuste a coluna que conte o CID.
tabelas['ANOMALIA_ID'] = tabelas.CODANOMAL_CID_UNICO.isna()
tabelas.ANOMALIA_ID.value_counts()

True     55624910
False      424518
Name: ANOMALIA_ID, dtype: int64

In [16]:
tabelas.head(3)

,IDADEMAE,GRAVIDEZ,PARTO,SEXO,PESO,IDANOMAL,ANO,FAIXA_ETARIA,CODANOMAL_CID_UNICO,ANOMALIA_ID
0,25.0,1.0,Normal,Masculino,3800.0,Não,2001,21-30,NaN,True
1,15.0,1.0,Normal,Masculino,3100.0,Não,2001,0-20,NaN,True
2,35.0,1.0,Cesária,Masculino,3300.0,Não,2001,31-40,NaN,True


In [17]:
# Na coluna 'ANOMALIA_ID' o valor 'False' indica que a coluna 'CODANOMAL_CID_UNICO' possui CID o que é contraditório com a
# coluna IDANOMAL que indica com o 'Não' que não foi idenficada anomalia.
# Assim vamos pegar os dados dessa coluna e obter seus index, para depois fazer o descarte
tabela_v1 = tabelas.query("IDANOMAL=='Não' & ANOMALIA_ID==False")
tabela_v1.head(3)

,IDADEMAE,GRAVIDEZ,PARTO,SEXO,PESO,IDANOMAL,ANO,FAIXA_ETARIA,CODANOMAL_CID_UNICO,ANOMALIA_ID
7443029,16.0,1.0,Normal,Feminino,2300.0,Não,2003,0-20,Q667,False
9048946,16.0,1.0,Normal,Masculino,3220.0,Não,2003,0-20,Q038,False
9050749,20.0,1.0,Cesária,Masculino,1955.0,Não,2003,0-20,Q432,False


In [18]:
lista_v1 = list(tabela_v1.index)
print('Total de Registros localizados: ',len(lista_v1))

Total de Registros localizados:  488


In [19]:
# Na coluna 'ANOMALIA_ID' o valor 'True' indica que a coluna 'CODANOMAL_CID_UNICO' não possui CID o que é contraditório com a
# coluna IDANOMAL que indica com o 'Sim' que foi idenficada anomalia.
# Assim vamos pegar os dados dessa coluna e obter seus index, para depois fazer o descarte
tabela_v2 = tabelas.query("IDANOMAL=='Sim' & ANOMALIA_ID==True")
tabela_v2.head(3)

,IDADEMAE,GRAVIDEZ,PARTO,SEXO,PESO,IDANOMAL,ANO,FAIXA_ETARIA,CODANOMAL_CID_UNICO,ANOMALIA_ID
1094331,41.0,1.0,Cesária,Feminino,3225.0,Sim,2001,41-50,NaN,True
2117824,17.0,1.0,Normal,Feminino,3000.0,Sim,2001,0-20,NaN,True
2118307,22.0,1.0,Normal,Masculino,3360.0,Sim,2001,21-30,NaN,True


In [20]:
lista_v2 = list(tabela_v2.index)
print('Total de Registros localizados: ',len(lista_v2))

Total de Registros localizados:  2579


In [21]:
# Deletentando Valores Inconsistentes da tabelas principal - Deletetando valores por linha usando os valores dos index
tabelas = tabelas.drop(lista_v1)
tabelas = tabelas.drop(lista_v2)

In [22]:
# Agora Vamos visualizar os totais novamente
#Total de Crianças com anomalias com codigo de Anomalia registrado na Coluna CODANOMAL
c_CID_anomalia = tabelas.CODANOMAL_CID_UNICO.value_counts().sum()
s_CID_anomalia = tabelas.CODANOMAL_CID_UNICO.value_counts(dropna=False)[0]
print(f'Total de Crianças com alguma anomalia: {c_CID_anomalia:,.0f}')
print(f'Total de Crianças sem anomalia       : {s_CID_anomalia:,.0f}')

Total de Crianças com alguma anomalia: 424,030
Total de Crianças sem anomalia       : 55,622,331


In [23]:
# Analise de Anomalias confirmadas com base na coluna IDANOMAL
c_anomalia = tabelas.IDANOMAL[tabelas.IDANOMAL=='Sim'].count()
s_anomalia = tabelas.IDANOMAL[tabelas.IDANOMAL=='Não'].count()
print(f'Total de Crianças com alguma anomalia: {c_anomalia:,.0f}')
print(f'Total de Crianças sem anomalia       : {s_anomalia:,.0f}')

Total de Crianças com alguma anomalia: 424,030
Total de Crianças sem anomalia       : 55,622,331


### 3.5 Fazendos os ajustes finais

In [24]:
# Deletando a coluna boleana usada para auxiliar na limpeza dos dados
tabelas = tabelas.drop(columns='ANOMALIA_ID')
tabelas.head(3)

,IDADEMAE,GRAVIDEZ,PARTO,SEXO,PESO,IDANOMAL,ANO,FAIXA_ETARIA,CODANOMAL_CID_UNICO
0,25.0,1.0,Normal,Masculino,3800.0,Não,2001,21-30,NaN
1,15.0,1.0,Normal,Masculino,3100.0,Não,2001,0-20,NaN
2,35.0,1.0,Cesária,Masculino,3300.0,Não,2001,31-40,NaN


In [25]:
# Alterando o nome das Colunas
colunas = {'IDADEMAE':'Idade_Mae',
           'GRAVIDEZ':'Numero_Filhos',
           'PARTO':'Tipo_Parto',
           'SEXO':'Sexo',
           'PESO':'Peso',
           'IDANOMAL':'ID_Anomalia',
           'ANO':'Ano',
           'FAIXA_ETARIA':'Faixa_Etaria',
           'CODANOMAL_CID_UNICO':'ID_CID'}
tabelas = tabelas.rename(columns=colunas)

In [26]:
tabelas.dtypes

Idade_Mae        float64
Numero_Filhos    float64
Tipo_Parto        object
Sexo              object
Peso             float64
ID_Anomalia       object
Ano                int64
Faixa_Etaria      object
ID_CID            object
dtype: object

In [27]:
# Separando os dados em 3 Partes: Sem Anomalia, Com Anomalia , Com Anomalia-SD
tabelas_com_anomalia_com_SD = tabelas.query("ID_CID=='Q900' or ID_CID=='Q901' or ID_CID=='Q902' or ID_CID=='Q909'")
tabelas_com_anomalia_sem_SD = tabelas.query("ID_Anomalia=='Sim'")
tabelas_sem_anomalia = tabelas.query("ID_Anomalia=='Não'")
# Construindo uma Nova Coluna para separa Anomalias com SD e sem SD.
tabelas_com_anomalia_com_SD['Situacao'] = 'Anomalia com SD'
tabelas_com_anomalia_sem_SD['Situacao'] = 'Anomalia sem SD'
tabelas_sem_anomalia['Situacao'] = 'Sem Anomalia'



In [28]:
# Eliminando valores duplicados da coluna tabelas_com_anomalia_sem_SD, pegando index da  com com SD
lista_v3 = list(tabelas_com_anomalia_com_SD.index)
print('Total de Registros localizados: ',len(lista_v3))

Total de Registros localizados:  17767


In [29]:
tabelas_com_anomalia_sem_SD = tabelas_com_anomalia_sem_SD.drop(lista_v3)
lista_v4 = list(tabelas_com_anomalia_sem_SD.index)
print('Total de Registros localizados apos o descarte: ',len(lista_v4))

Total de Registros localizados apos o descarte:  406263


In [30]:
#Reagrupando os Dados
tabelas = pd.concat([tabelas_sem_anomalia,tabelas_com_anomalia_com_SD,tabelas_com_anomalia_sem_SD])

In [31]:
# Substituindo os valores nulos da coluna ID_CID por Não, par aos caso em que não foi indenficada anomalia, 
# com base na informação da coluna ID_Anomalia.
tabelas.ID_CID = tabelas.ID_CID.fillna('Não')

In [32]:
tabelas.ID_CID.value_counts().head(3)

Não     55622331
Q699       35180
Q668       23496
Name: ID_CID, dtype: int64

In [33]:
# Convertendo a coluna Ano para String
tabelas['Ano'] = tabelas['Ano'].astype(str)

In [34]:
# Verificando os dados após as modificações
tabelas.ID_Anomalia.value_counts()

Não    55622331
Sim      424030
Name: ID_Anomalia, dtype: int64

In [35]:
# Verificando a coluna nova criada
tabelas.Situacao.value_counts()

Sem Anomalia       55622331
Anomalia sem SD      406263
Anomalia com SD       17767
Name: Situacao, dtype: int64

In [36]:
tabelas.Situacao.value_counts(normalize=1)*100

Sem Anomalia       99.243430
Anomalia sem SD     0.724870
Anomalia com SD     0.031701
Name: Situacao, dtype: float64

In [37]:
# Visualizando a tabela final
tabelas

,Idade_Mae,Numero_Filhos,Tipo_Parto,Sexo,Peso,ID_Anomalia,Ano,Faixa_Etaria,ID_CID,Situacao
0,25.0,1.0,Normal,Masculino,3800.0,Não,2001,21-30,Não,Sem Anomalia
1,15.0,1.0,Normal,Masculino,3100.0,Não,2001,0-20,Não,Sem Anomalia
2,35.0,1.0,Cesária,Masculino,3300.0,Não,2001,31-40,Não,Sem Anomalia
3,17.0,1.0,Normal,Feminino,3200.0,Não,2001,0-20,Não,Sem Anomalia
6,24.0,1.0,Normal,Masculino,3750.0,Não,2001,21-30,Não,Sem Anomalia
...,...,...,...,...,...,...,...,...,...,...
58770629,16.0,1.0,Cesária,Masculino,2440.0,Sim,2020,0-20,Q793,Anomalia sem SD
58770643,21.0,1.0,Normal,Masculino,3375.0,Sim,2020,21-30,Q833,Anomalia sem SD
58771138,21.0,1.0,Cesária,Feminino,2714.0,Sim,2020,21-30,Q848,Anomalia sem SD
58771155,40.0,1.0,Cesária,Feminino,3335.0,Sim,2020,31-40,Q181,Anomalia sem SD


In [38]:
# Salvando os fabela final
tabelas.to_csv('tabelas_v3.csv', index=False)